# Section 1

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.transforms import RandomNodeSplit
from torch_geometric.utils import negative_sampling
import matplotlib.pyplot as plt

In [7]:
# Load the dataset
file_path = '/home/arifai/hussain/Race_pathways_signatures_meta_cleaned.csv'
data = pd.read_csv(file_path)

# Remove columns with repeated names
data = data.loc[:, ~data.columns.duplicated()]

# Assign binary labels based on race
data['label'] = data['Race'].apply(lambda x: 0 if x == "White (Non-Hispanic)" else 1 if x == "African American" else np.nan)

# Drop rows with NaN labels (if any)
data = data.dropna(subset=['label'])

# Columns to remove
columns_to_remove = [
    'pathgs', 'pathgs_p', 'pathgs_s', 'Race', 'svi', 'sm', 'lni', 'epe', 'avg.risk', 'Decipher', 'APF', 'age',
    'hallmark_adipogenesis', 'hallmark_allograft_rejection', 'hallmark_androgen_response', 'hallmark_angiogenesis',
    'hallmark_angiogenesis_Brauer2013', 'hallmark_angiogenesis_KeggVEGF', 'hallmark_angiogenesis_Liberzon2015',
    'hallmark_angiogenesis_Masiero2013', 'hallmark_angiogenesis_Nolan2013', 'hallmark_angiogenesis_Uhlik2016',
    'hallmark_apical_junction', 'hallmark_apical_surface', 'hallmark_apoptosis', 'hallmark_bile_acid_metabolism',
    'hallmark_cholesterol_homeostasis', 'hallmark_coagulation', 'hallmark_complement', 'hallmark_dna_repair',
    'hallmark_e2f_targets', 'hallmark_epithelial_mesenchymal_transition', 'hallmark_estrogen_response_early',
    'hallmark_estrogen_response_late', 'hallmark_fatty_acid_metabolism', 'hallmark_g2m_checkpoint', 'hallmark_glycolysis',
    'hallmark_hedgehog_signaling', 'hallmark_heme_metabolism', 'hallmark_hypoxia', 'hallmark_il2_stat5_signaling',
    'hallmark_il6_jak_stat3_signaling', 'hallmark_inflammatory_response', 'hallmark_interferon_alpha_response',
    'hallmark_interferon_gamma_response', 'hallmark_kras_signaling_dn', 'hallmark_kras_signaling_up', 'hallmark_mitotic_spindle',
    'hallmark_mtorc1_signaling', 'hallmark_myc_targets_v1', 'hallmark_myc_targets_v2', 'hallmark_myogenesis',
    'hallmark_notch_signaling', 'hallmark_oxidative_phosphorylation', 'hallmark_p53_pathway', 'hallmark_pancreas_beta_cells',
    'hallmark_peroxisome', 'hallmark_pi3k_akt_mtor_signaling', 'hallmark_protein_secretion', 'hallmark_reactive_oxigen_species_pathway',
    'hallmark_spermatogenesis', 'hallmark_tgf_beta_signaling', 'hallmark_tnfa_signaling_via_nfkb', 'hallmark_unfolded_protein_response',
    'hallmark_uv_response_dn', 'hallmark_uv_response_up', 'hallmark_wnt_beta_catenin_signaling', 'hallmark_xenobiotic_metabolism',
    'agell2012_1', 'cheville2008_1', 'cuzick2011_1', 'decipher_1', 'glinsky2005_1', 'klein2014_1', 'lapointe2004_1', 'larkin2012_1',
    'long2014_1', 'nakagawa2008_1', 'penney2011_1', 'ramaswamy2003_1', 'saal2007_1', 'singh2002_1', 'stephenson2005_1', 'talantov2010_1',
    'varambally2005_1', 'wu2013_1', 'yu2007_1'
]

# Remove specified columns
data = data.drop(columns=columns_to_remove)

# Drop non-numeric columns (assuming the first 3 columns are metadata)
gene_expression = data.iloc[:, 3:-1]  # Exclude the label column

# Handle missing values (e.g., fill with the mean of the column)
gene_expression = gene_expression.apply(pd.to_numeric, errors='coerce')
gene_expression = gene_expression.fillna(gene_expression.mean())

# Convert to numpy array
gene_expression = gene_expression.values

# Compute correlation matrix
correlation_matrix = np.corrcoef(gene_expression, rowvar=False)

# Define a threshold for creating edges
threshold = 0.7
edges = np.where(np.abs(correlation_matrix) > threshold)
edge_index = torch.tensor(edges, dtype=torch.long)

# Create node features (gene expression levels)
x = torch.tensor(gene_expression, dtype=torch.float)

# Use actual labels from the dataset
y = torch.tensor(data['label'].values, dtype=torch.long)

# Create PyG data object
data = Data(x=x, edge_index=edge_index, y=y)

# Manually split the data into training and test sets
num_train = 1000
num_test = 152

# Create masks for training and test sets
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[:num_train] = True
test_mask[num_train:num_train + num_test] = True

data.train_mask = train_mask
data.test_mask = test_mask

# Define GAT model for node classification
class GATNodeClassification(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_heads=8):
        super(GATNodeClassification, self).__init__()
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv3 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv4 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, dropout=0.6)
        self.fc = torch.nn.Linear(hidden_channels * num_heads, 2)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Initialize model, loss, and optimizer
model = GATNodeClassification(num_node_features=data.num_features, hidden_channels=256)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# Training loop for node classification
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss.item()

# Evaluation for node classification
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = int(correct) / int(data.test_mask.sum())
        return acc

# Store loss and accuracy for plotting
train_losses = []
test_accuracies = []

# Train the model
for epoch in range(200):
    loss = train()
    acc = test()
    train_losses.append(loss)
    test_accuracies.append(acc)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')

# Save the model
torch.save(model.state_dict(), '/root/gat_node_classification_model.pth')

# Load the model
model = GATNodeClassification(num_node_features=data.num_features, hidden_channels=256)
model.load_state_dict(torch.load('/root/gat_node_classification_model.pth'))
model.eval()

# Print the final test accuracy
final_test_accuracy = test()
print(f'Final Test Accuracy: {final_test_accuracy}')

# Plotting the results
epochs = range(200)

plt.figure(figsize=(12, 5))

# Plot Training Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

# Plot Test Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.savefig('/root/node_classification_loss_accuracy.png')
plt.show()


/tmp/ipykernel_4161/3794757535.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  edge_index = torch.tensor(edges, dtype=torch.long)


Epoch 0, Loss: 0.81682288646698, Accuracy: 0.4473684210526316
Epoch 10, Loss: 0.7875545620918274, Accuracy: 0.9868421052631579


KeyboardInterrupt: 

# Section 2

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import torch_geometric
from torch_geometric.utils import to_networkx

# Use the preprocessed data from Section 1
# Compute correlation matrix
correlation_matrix = np.corrcoef(gene_expression, rowvar=False)

print(np.shape(correlation_matrix))

# Define a threshold for creating edges
threshold = 0.7
edges = np.where((np.abs(correlation_matrix) > threshold) & (np.abs(correlation_matrix != 1)))
edge_index = torch.tensor(edges, dtype=torch.long)

print(np.shape(edges))

# Create a NetworkX graph
G = nx.Graph()

# Add nodes (genes)
num_genes = correlation_matrix.shape[0]
G.add_nodes_from(range(num_genes))

# Add edges based on the correlation threshold
for i in range(edges[0].shape[0]):
    G.add_edge(edges[0][i], edges[1][i])

# Plot the graph
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42)  # Position nodes using the spring layout
nx.draw(G, pos, with_labels=True, node_size=50, node_color='skyblue', edge_color='gray', font_size=8)
plt.title('Gene Interaction Network')
plt.show()


# Section 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import community as community_louvain
import torch_geometric

# Load the dataset
file_path = '/root/Race_pathways_signatures_meta_cleaned.csv'
df = pd.read_csv(file_path)

# Remove columns with repeated names
df = df.loc[:, ~df.columns.duplicated()]

# Assign binary labels based on race
df['label'] = df['Race'].apply(lambda x: 0 if x == "White (Non-Hispanic)" else 1 if x == "African American" else np.nan)

# Drop rows with NaN labels (if any)
df = df.dropna(subset=['label'])

# Columns to remove
columns_to_remove = [
    'pathgs', 'pathgs_p', 'pathgs_s', 'Race', 'svi', 'sm', 'lni', 'epe', 'avg.risk', 'Decipher', 'APF', 'age',
    'hallmark_adipogenesis', 'hallmark_allograft_rejection', 'hallmark_androgen_response', 'hallmark_angiogenesis',
    'hallmark_angiogenesis_Brauer2013', 'hallmark_angiogenesis_KeggVEGF', 'hallmark_angiogenesis_Liberzon2015',
    'hallmark_angiogenesis_Masiero2013', 'hallmark_angiogenesis_Nolan2013', 'hallmark_angiogenesis_Uhlik2016',
    'hallmark_apical_junction', 'hallmark_apical_surface', 'hallmark_apoptosis', 'hallmark_bile_acid_metabolism',
    'hallmark_cholesterol_homeostasis', 'hallmark_coagulation', 'hallmark_complement', 'hallmark_dna_repair',
    'hallmark_e2f_targets', 'hallmark_epithelial_mesenchymal_transition', 'hallmark_estrogen_response_early',
    'hallmark_estrogen_response_late', 'hallmark_fatty_acid_metabolism', 'hallmark_g2m_checkpoint', 'hallmark_glycolysis',
    'hallmark_hedgehog_signaling', 'hallmark_heme_metabolism', 'hallmark_hypoxia', 'hallmark_il2_stat5_signaling',
    'hallmark_il6_jak_stat3_signaling', 'hallmark_inflammatory_response', 'hallmark_interferon_alpha_response',
    'hallmark_interferon_gamma_response', 'hallmark_kras_signaling_dn', 'hallmark_kras_signaling_up', 'hallmark_mitotic_spindle',
    'hallmark_mtorc1_signaling', 'hallmark_myc_targets_v1', 'hallmark_myc_targets_v2', 'hallmark_myogenesis',
    'hallmark_notch_signaling', 'hallmark_oxidative_phosphorylation', 'hallmark_p53_pathway', 'hallmark_pancreas_beta_cells',
    'hallmark_peroxisome', 'hallmark_pi3k_akt_mtor_signaling', 'hallmark_protein_secretion', 'hallmark_reactive_oxigen_species_pathway',
    'hallmark_spermatogenesis', 'hallmark_tgf_beta_signaling', 'hallmark_tnfa_signaling_via_nfkb', 'hallmark_unfolded_protein_response',
    'hallmark_uv_response_dn', 'hallmark_uv_response_up', 'hallmark_wnt_beta_catenin_signaling', 'hallmark_xenobiotic_metabolism',
    'agell2012_1', 'cheville2008_1', 'cuzick2011_1', 'decipher_1', 'glinsky2005_1', 'klein2014_1', 'lapointe2004_1', 'larkin2012_1',
    'long2014_1', 'nakagawa2008_1', 'penney2011_1', 'ramaswamy2003_1', 'saal2007_1', 'singh2002_1', 'stephenson2005_1', 'talantov2010_1',
    'varambally2005_1', 'wu2013_1', 'yu2007_1'
]

# Remove specified columns
df = df.drop(columns=columns_to_remove)

# Extract gene names from the original DataFrame
gene_names = df.columns[3:-1]

# Drop non-numeric columns (assuming the first 3 columns are metadata)
gene_expression = df.iloc[:, 3:-1].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in gene expression data and update labels
gene_expression = gene_expression.dropna()
labels = df['label'].loc[gene_expression.index].values

# Function to get top interactions
def get_top_interactions(gene_expression, labels, race_label, top_n=5):
    race_indices = np.where(labels == race_label)[0]
    race_corr_matrix = np.corrcoef(gene_expression.iloc[race_indices].values, rowvar=False)
    np.fill_diagonal(race_corr_matrix, 0)
    strongest_interactions = np.unravel_index(np.argsort(-np.abs(race_corr_matrix), axis=None), race_corr_matrix.shape)
    top_interactions = []
    seen_pairs = set()
    for i in range(len(strongest_interactions[0])):
        gene1, gene2 = strongest_interactions[0][i], strongest_interactions[1][i]
        if (gene1, gene2) not in seen_pairs and (gene2, gene1) not in seen_pairs and gene1 < len(gene_names) and gene2 < len(gene_names) and gene_names[gene1] != gene_names[gene2]:
            top_interactions.append((gene1, gene2))
            seen_pairs.add((gene1, gene2))
            if len(top_interactions) == top_n:
                break
    return top_interactions

# Function to visualize the correlation matrix
def visualize_correlation_matrix(correlation_matrix, title):
    plt.figure(figsize=(10, 8))
    plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
    plt.colorbar()
    plt.xticks(ticks=np.arange(len(gene_names)), labels=gene_names, rotation=90, fontsize=3)
    plt.yticks(ticks=np.arange(len(gene_names)), labels=gene_names, fontsize=3)
    plt.title(title)
    plt.show()

# Function to perform topological analysis
def topological_analysis(gene_expression, labels, race_label, race_name):
    race_indices = np.where(labels == race_label)[0]
    race_corr_matrix = np.corrcoef(gene_expression.iloc[race_indices].values, rowvar=False)
    np.fill_diagonal(race_corr_matrix, 0)
    
    # Construct the gene co-expression network
    G = nx.Graph()
    for gene in gene_names:
        G.add_node(gene)
    for i, gene1 in enumerate(gene_names):
        for j, gene2 in enumerate(gene_names):
            if i < j and abs(race_corr_matrix[i, j]) > 0.7:
                G.add_edge(gene1, gene2, weight=race_corr_matrix[i, j])
    
    # Degree centrality
    degree_centrality = nx.degree_centrality(G)
    print("Degree Centrality:", degree_centrality)
    
    # Betweenness centrality
    betweenness_centrality = nx.betweenness_centrality(G)
    print("Betweenness Centrality:", betweenness_centrality)
    
    # Closeness centrality
    closeness_centrality = nx.closeness_centrality(G)
    print("Closeness Centrality:", closeness_centrality)
    
    # Eigenvector centrality
    eigenvector_centrality = nx.eigenvector_centrality(G)
    print("Eigenvector Centrality:", eigenvector_centrality)
    
    # Clustering coefficient
    clustering_coefficient = nx.clustering(G)
    print("Clustering Coefficient:", clustering_coefficient)
    
    # Community detection using the Louvain method
    partition = community_louvain.best_partition(G)
    print("Community Detection (Louvain):", partition)
    
    # Visualize the network with community detection
    pos = nx.spring_layout(G)
    cmap = plt.get_cmap('viridis')
    nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40, cmap=cmap, node_color=list(partition.values()))
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    plt.title(f"Gene Co-expression Network for {race_name}")
    plt.show()

# Get top 5 interactions for African Americans (label 1)
top_interactions_aa = get_top_interactions(gene_expression, labels, 1)
print("Top 5 strongest interactions for African Americans:")
for i, (gene1, gene2) in enumerate(top_interactions_aa):
    print(f"Interaction {i+1}: {gene_names[gene1]} - {gene_names[gene2]}")

# Visualize the correlation matrix for African Americans
race_indices_aa = np.where(labels == 1)[0]
race_corr_matrix_aa = np.corrcoef(gene_expression.iloc[race_indices_aa].values, rowvar=False)
visualize_correlation_matrix(race_corr_matrix_aa, "Correlation Matrix for African Americans")

# Perform topological analysis for African Americans
topological_analysis(gene_expression, labels, 1, "African Americans")

# Get top 5 interactions for White (Non-Hispanic) (label 0)
top_interactions_wh = get_top_interactions(gene_expression, labels, 0)
print("Top 5 strongest interactions for White (Non-Hispanic):")
for i, (gene1, gene2) in enumerate(top_interactions_wh):
    print(f"Interaction {i+1}: {gene_names[gene1]} - {gene_names[gene2]}")

# Visualize the correlation matrix for White (Non-Hispanic)
race_indices_wh = np.where(labels == 0)[0]
race_corr_matrix_wh = np.corrcoef(gene_expression.iloc[race_indices_wh].values, rowvar=False)
visualize_correlation_matrix(race_corr_matrix_wh, "Correlation Matrix for White (Non-Hispanic)")

# Perform topological analysis for White (Non-Hispanic)
topological_analysis(gene_expression, labels, 0, "White (Non-Hispanic)")

# Section 4

In [ ]:
############################# GCN NODE CLASSIFICATION AND PREDICTION SECTION
########################### (loss: the loss of current interations)
############################# (accuracy: rate of properly classifying race per node)

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/root/Race_pathways_signatures_meta_cleaned.csv'
data = pd.read_csv(file_path)

# Remove columns with repeated names
data = data.loc[:, ~data.columns.duplicated()]

# Assign binary labels based on race
data['label'] = data['Race'].apply(lambda x: 0 if x == "White (Non-Hispanic)" else 1 if x == "African American" else np.nan)

# Drop rows with NaN labels (if any)
data = data.dropna(subset=['label'])

# Columns to remove
columns_to_remove = [
    'pathgs', 'pathgs_p', 'pathgs_s', 'Race', 'svi', 'sm', 'lni', 'epe', 'avg.risk', 'Decipher', 'APF', 'age',
    'hallmark_adipogenesis', 'hallmark_allograft_rejection', 'hallmark_androgen_response', 'hallmark_angiogenesis',
    'hallmark_angiogenesis_Brauer2013', 'hallmark_angiogenesis_KeggVEGF', 'hallmark_angiogenesis_Liberzon2015',
    'hallmark_angiogenesis_Masiero2013', 'hallmark_angiogenesis_Nolan2013', 'hallmark_angiogenesis_Uhlik2016',
    'hallmark_apical_junction', 'hallmark_apical_surface', 'hallmark_apoptosis', 'hallmark_bile_acid_metabolism',
    'hallmark_cholesterol_homeostasis', 'hallmark_coagulation', 'hallmark_complement', 'hallmark_dna_repair',
    'hallmark_e2f_targets', 'hallmark_epithelial_mesenchymal_transition', 'hallmark_estrogen_response_early',
    'hallmark_estrogen_response_late', 'hallmark_fatty_acid_metabolism', 'hallmark_g2m_checkpoint', 'hallmark_glycolysis',
    'hallmark_hedgehog_signaling', 'hallmark_heme_metabolism', 'hallmark_hypoxia', 'hallmark_il2_stat5_signaling',
    'hallmark_il6_jak_stat3_signaling', 'hallmark_inflammatory_response', 'hallmark_interferon_alpha_response',
    'hallmark_interferon_gamma_response', 'hallmark_kras_signaling_dn', 'hallmark_kras_signaling_up', 'hallmark_mitotic_spindle',
    'hallmark_mtorc1_signaling', 'hallmark_myc_targets_v1', 'hallmark_myc_targets_v2', 'hallmark_myogenesis',
    'hallmark_notch_signaling', 'hallmark_oxidative_phosphorylation', 'hallmark_p53_pathway', 'hallmark_pancreas_beta_cells',
    'hallmark_peroxisome', 'hallmark_pi3k_akt_mtor_signaling', 'hallmark_protein_secretion', 'hallmark_reactive_oxigen_species_pathway',
    'hallmark_spermatogenesis', 'hallmark_tgf_beta_signaling', 'hallmark_tnfa_signaling_via_nfkb', 'hallmark_unfolded_protein_response',
    'hallmark_uv_response_dn', 'hallmark_uv_response_up', 'hallmark_wnt_beta_catenin_signaling', 'hallmark_xenobiotic_metabolism',
    'agell2012_1', 'cheville2008_1', 'cuzick2011_1', 'decipher_1', 'glinsky2005_1', 'klein2014_1', 'lapointe2004_1', 'larkin2012_1',
    'long2014_1', 'nakagawa2008_1', 'penney2011_1', 'ramaswamy2003_1', 'saal2007_1', 'singh2002_1', 'stephenson2005_1', 'talantov2010_1',
    'varambally2005_1', 'wu2013_1', 'yu2007_1'
]

# Remove specified columns
data = data.drop(columns=columns_to_remove)

# Drop non-numeric columns (assuming the first 3 columns are metadata)
gene_expression = data.iloc[:, 3:-1]  # Exclude the label column

# Handle missing values (e.g., fill with the mean of the column)
gene_expression = gene_expression.apply(pd.to_numeric, errors='coerce')
gene_expression = gene_expression.fillna(gene_expression.mean())

# Convert to numpy array
gene_expression = gene_expression.values

# Compute correlation matrix
correlation_matrix = np.corrcoef(gene_expression, rowvar=False)

# Define a threshold for creating edges
threshold = 0.7
edges = np.where(np.abs(correlation_matrix) > threshold)
edges = np.array(edges)  # Convert to a single numpy array
edge_index = torch.tensor(edges, dtype=torch.long)

# Create node features (gene expression levels)
x = torch.tensor(gene_expression, dtype=torch.float)

# Use actual labels from the dataset
y = torch.tensor(data['label'].values, dtype=torch.long)

# Create PyG data object
data = Data(x=x, edge_index=edge_index, y=y)

# Manually split the data into training and test sets
num_train = 1000
num_test = 152

# Create masks for training and test sets
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[:num_train] = True
test_mask[num_train:num_train + num_test] = True

data.train_mask = train_mask
data.test_mask = test_mask

# Define GNN model for node classification
class GCNNodeClassification(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCNNodeClassification, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.bn3 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.bn4 = torch.nn.BatchNorm1d(hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, 2)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Initialize model, loss, and optimizer
model = GCNNodeClassification(num_node_features=data.num_features, hidden_channels=256)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# Training loop for node classification
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss.item()

# Evaluation for node classification
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = int(correct) / int(data.test_mask.sum())
        return acc

# Store loss and accuracy for plotting
train_losses = []
test_accuracies = []

# Train the model
for epoch in range(200):
    loss = train()
    acc = test()
    train_losses.append(loss)
    test_accuracies.append(acc)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')

# Save the model
torch.save(model.state_dict(), '/root/gcn_node_classification_model.pth')

# Load the model
model = GCNNodeClassification(num_node_features=data.num_features, hidden_channels=256)
model.load_state_dict(torch.load('/root/gcn_node_classification_model.pth'))
model.eval()

# Print the final test accuracy
final_test_accuracy = test()
print(f'Final Test Accuracy: {final_test_accuracy}')

# Plotting the results for node classification
epochs = range(200)

plt.figure(figsize=(12, 5))

# Plot Training Loss for Node Classification
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss for Node Classification')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss for Node Classification over Epochs')
plt.legend()

# Plot Test Accuracy for Node Classification
plt.subplot(1, 2, 2)
plt.plot(epochs, test_accuracies, label='Test Accuracy for Node Classification')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy for Node Classification over Epochs')
plt.legend()

plt.tight_layout()
plt.savefig('/root/node_classification_loss_accuracy.png')
plt.show()


# Section 5

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/root/Race_pathways_signatures_meta_cleaned.csv'
data = pd.read_csv(file_path)

# Remove columns with repeated names
data = data.loc[:, ~data.columns.duplicated()]

# Assign binary labels based on race
data['label'] = data['Race'].apply(lambda x: 0 if x == "White (Non-Hispanic)" else 1 if x == "African American" else np.nan)

# Drop rows with NaN labels (if any)
data = data.dropna(subset=['label'])

# Columns to remove
columns_to_remove = [
    'pathgs', 'pathgs_p', 'pathgs_s', 'Race', 'svi', 'sm', 'lni', 'epe', 'avg.risk', 'Decipher', 'APF', 'age',
    'hallmark_adipogenesis', 'hallmark_allograft_rejection', 'hallmark_androgen_response', 'hallmark_angiogenesis',
    'hallmark_angiogenesis_Brauer2013', 'hallmark_angiogenesis_KeggVEGF', 'hallmark_angiogenesis_Liberzon2015',
    'hallmark_angiogenesis_Masiero2013', 'hallmark_angiogenesis_Nolan2013', 'hallmark_angiogenesis_Uhlik2016',
    'hallmark_apical_junction', 'hallmark_apical_surface', 'hallmark_apoptosis', 'hallmark_bile_acid_metabolism',
    'hallmark_cholesterol_homeostasis', 'hallmark_coagulation', 'hallmark_complement', 'hallmark_dna_repair',
    'hallmark_e2f_targets', 'hallmark_epithelial_mesenchymal_transition', 'hallmark_estrogen_response_early',
    'hallmark_estrogen_response_late', 'hallmark_fatty_acid_metabolism', 'hallmark_g2m_checkpoint', 'hallmark_glycolysis',
    'hallmark_hedgehog_signaling', 'hallmark_heme_metabolism', 'hallmark_hypoxia', 'hallmark_il2_stat5_signaling',
    'hallmark_il6_jak_stat3_signaling', 'hallmark_inflammatory_response', 'hallmark_interferon_alpha_response',
    'hallmark_interferon_gamma_response', 'hallmark_kras_signaling_dn', 'hallmark_kras_signaling_up', 'hallmark_mitotic_spindle',
    'hallmark_mtorc1_signaling', 'hallmark_myc_targets_v1', 'hallmark_myc_targets_v2', 'hallmark_myogenesis',
    'hallmark_notch_signaling', 'hallmark_oxidative_phosphorylation', 'hallmark_p53_pathway', 'hallmark_pancreas_beta_cells',
    'hallmark_peroxisome', 'hallmark_pi3k_akt_mtor_signaling', 'hallmark_protein_secretion', 'hallmark_reactive_oxigen_species_pathway',
    'hallmark_spermatogenesis', 'hallmark_tgf_beta_signaling', 'hallmark_tnfa_signaling_via_nfkb', 'hallmark_unfolded_protein_response',
    'hallmark_uv_response_dn', 'hallmark_uv_response_up', 'hallmark_wnt_beta_catenin_signaling', 'hallmark_xenobiotic_metabolism',
    'agell2012_1', 'cheville2008_1', 'cuzick2011_1', 'decipher_1', 'glinsky2005_1', 'klein2014_1', 'lapointe2004_1', 'larkin2012_1',
    'long2014_1', 'nakagawa2008_1', 'penney2011_1', 'ramaswamy2003_1', 'saal2007_1', 'singh2002_1', 'stephenson2005_1', 'talantov2010_1',
    'varambally2005_1', 'wu2013_1', 'yu2007_1'
]

# Remove specified columns
data = data.drop(columns=columns_to_remove)

# Drop non-numeric columns (assuming the first 3 columns are metadata)
gene_expression = data.iloc[:, 3:-1]  # Exclude the label column

# Handle missing values (e.g., fill with the mean of the column)
gene_expression = gene_expression.apply(pd.to_numeric, errors='coerce')
gene_expression = gene_expression.fillna(gene_expression.mean())

# Convert to numpy array
gene_expression = gene_expression.values

# Compute correlation matrix
correlation_matrix = np.corrcoef(gene_expression, rowvar=False)

# Define a threshold for creating edges
threshold = 0.7
edges = np.where(np.abs(correlation_matrix) > threshold)

# Convert the list of numpy arrays to a single numpy array
edges = np.array(edges)

# Create edge_index tensor
edge_index = torch.tensor(edges, dtype=torch.long)

# Create node features (gene expression levels)
x = torch.tensor(gene_expression, dtype=torch.float)

# Use actual labels from the dataset
y = torch.tensor(data['label'].values, dtype=torch.long)

# Create PyG data object
data = Data(x=x, edge_index=edge_index, y=y)

# Generate negative samples for link prediction
negative_edge_index = negative_sampling(edge_index, num_neg_samples=edge_index.size(1))

# Combine positive and negative edges for training
train_edge_index = torch.cat([edge_index, negative_edge_index], dim=1)

# Create edge labels (1 for positive edges, 0 for negative edges)
train_edge_labels = torch.cat([torch.ones(edge_index.size(1)), torch.zeros(negative_edge_index.size(1))], dim=0).unsqueeze(1)

# Define GNN model for node classification
class GCNNodeClassification(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCNNodeClassification, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.bn3 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.bn4 = torch.nn.BatchNorm1d(hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, 2)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.dropout(x)
        return x

# Define MLP model for link prediction
class MLPLinkPrediction(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(MLPLinkPrediction, self).__init__()
        self.gcn = GCNNodeClassification(num_node_features, hidden_channels)
        self.fc1 = torch.nn.Linear(hidden_channels * 2, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, 1)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x, edge_index, edge_index_pos, edge_index_neg):
        x = self.gcn(x, edge_index)
        pos_edge_embeddings = torch.cat([x[edge_index_pos[0]], x[edge_index_pos[1]]], dim=1)
        neg_edge_embeddings = torch.cat([x[edge_index_neg[0]], x[edge_index_neg[1]]], dim=1)
        edge_embeddings = torch.cat([pos_edge_embeddings, neg_edge_embeddings], dim=0)
        x = F.relu(self.fc1(edge_embeddings))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer for link prediction
model_link_pred = MLPLinkPrediction(num_node_features=data.num_features, hidden_channels=256)
criterion_link_pred = torch.nn.BCEWithLogitsLoss()
optimizer_link_pred = torch.optim.Adam(model_link_pred.parameters(), lr=0.001)

# Training loop for link prediction
def train_link_prediction():
    model_link_pred.train()
    optimizer_link_pred.zero_grad()
    out = model_link_pred(data.x, data.edge_index, edge_index, negative_edge_index)
    loss = criterion_link_pred(out, train_edge_labels)
    loss.backward()
    optimizer_link_pred.step()
    return loss.item()

# Evaluation for link prediction
def test_link_prediction():
    model_link_pred.eval()
    with torch.no_grad():
        out = model_link_pred(data.x, data.edge_index, edge_index, negative_edge_index)
        pred = torch.sigmoid(out)
        pred_labels = (pred > 0.5).float()
        correct = (pred_labels == train_edge_labels).sum()
        acc = int(correct) / len(train_edge_labels)
        return acc

# Store loss and accuracy for plotting
train_losses_link_pred = []
test_accuracies_link_pred = []

# Train the link prediction model
for epoch in range(200):
    loss = train_link_prediction()
    acc = test_link_prediction()
    train_losses_link_pred.append(loss)
    test_accuracies_link_pred.append(acc)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Link Prediction Loss: {loss}, Link Prediction Accuracy: {acc}')

# Save the link prediction model
torch.save(model_link_pred.state_dict(), '/root/mlp_link_prediction_model.pth')

# Load the link prediction model
model_link_pred = MLPLinkPrediction(num_node_features=data.num_features, hidden_channels=256)
model_link_pred.load_state_dict(torch.load('/root/mlp_link_prediction_model.pth'))
model_link_pred.eval()

# Print the final test accuracy for link prediction
final_test_accuracy_link_pred = test_link_prediction()
print(f'Final Test Accuracy for Link Prediction: {final_test_accuracy_link_pred}')

# Plotting the results for link prediction
epochs = range(200)

plt.figure(figsize=(12, 5))

# Plot Training Loss for Link Prediction
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses_link_pred, label='Training Loss for Link Prediction')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss for Link Prediction over Epochs')
plt.legend()

# Plot Test Accuracy for Link Prediction
plt.subplot(1, 2, 2)
plt.plot(epochs, test_accuracies_link_pred, label='Test Accuracy for Link Prediction')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy for Link Prediction over Epochs')
plt.legend()

plt.tight_layout()
plt.savefig('/root/link_prediction_loss_accuracy.png')
plt.show()